In [1]:
import pandas as pd

In [7]:
df = pd.read_csv(r'Data/scopus_bd_aumentada.csv', sep=",")

C:\Users\YORLEY\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (7,21,22,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
df.columns

Index(['Authors', 'Author(s)_ID', 'Authors_Rev', 'Author(s)_ID_Rev', 'Title',
       'Year', 'Source_title', 'Volume', 'Issue', 'Art._No.', 'Page_start',
       'Page_end', 'Page_count', 'Cited_by', 'DOI', 'Link', 'Affiliations',
       'Authors_with_affiliations', 'Abstract', 'Author_Keywords',
       'Index_Keywords', 'ISSN', 'ISBN', 'CODEN', 'PubMed_ID', 'Document_Type',
       'Publication_Stage', 'Access_Type', 'Source', 'EID'],
      dtype='object')

In [13]:
df.head(5)

,Authors,Author(s)_ID,Authors_Rev,Author(s)_ID_Rev,Title,Year,Source_title,Volume,Issue,Art._No.,...,Index_Keywords,ISSN,ISBN,CODEN,PubMed_ID,Document_Type,Publication_Stage,Access_Type,Source,EID
0,Vanzolini P.E.,6602373241,Vanzolini P.E.,6602373241,"On Gymnodactylus amarali Barbour, 1925, with t...",2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,...,animal; article; Brazil; classification; compa...,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044450573
1,De Wagener A.L.R.,7004422261,De Wagener A.L.R.,7004422261,Constraints to the implementation of effective...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,...,article; developing country; environmental mon...,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044447222
2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...",11739855600;7202349172;11738879800;6603689065;...,Mexias A.S.,11739855600,Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,...,article; Butia,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319
3,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...",11739855600;7202349172;11738879800;6603689065;...,Berger G.,7202349172,Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,...,article; Butia,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319
4,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...",11739855600;7202349172;11738879800;6603689065;...,Gomes M.E.B.,11738879800,Geochemical modeling of gold precipitation con...,2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,...,article; Butia,13765,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044441319


In [19]:
df_authors = df[['Authors','Author(s)_ID_Rev']][:].drop_duplicates()

In [20]:
len(df_authors)

171818

In [22]:
df_authors.head(10)

,Authors,Author(s)_ID_Rev
0,Vanzolini P.E.,6602373241
1,De Wagener A.L.R.,7004422261
2,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...",11739855600
3,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...",7202349172
4,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...",11738879800
5,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...",6603689065
6,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...",6507369527
7,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...",11739533300
8,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...",11739290800
9,Mulloy B.,55179052500


### Total Citaciones

In [25]:
df_temp=df[['Author(s)_ID_Rev', 'Cited_by']].groupby(by='Author(s)_ID_Rev').sum()
df_temp.reset_index(level=0, inplace=True)
df_temp.head()

,Author(s)_ID_Rev,Cited_by
0,1,0.0
1,10038943700,12.0
2,10039167100,9.0
3,10039295200,19.0
4,10039317900,22.0


In [29]:
df_authors = df_authors.merge(right=df_temp, how='inner', on='Author(s)_ID_Rev')

### Total Artículos publicados

In [31]:
df_authors.head()

,Authors,Author(s)_ID_Rev,Cited_by
0,Vanzolini P.E.,6602373241,62.0
1,"Vanzolini P.E., Calleffo M.E.V.",6602373241,62.0
2,De Wagener A.L.R.,7004422261,6.0
3,"Mexias A.S., Berger G., Gomes M.E.B., Formoso ...",11739855600,11.0
4,"Bongiolo E.M., Bongiolo D.E., Sardini P., Mexi...",11739855600,11.0


In [35]:
df_temp=df[['Author(s)_ID_Rev', 'Title']].groupby(by='Author(s)_ID_Rev').value_counts()
df_temp.reset_index(level=0, inplace=True)
df_temp.head()

AttributeError: 'DataFrameGroupBy' object has no attribute 'value_counts'